> 📦 **Note:** Before running this notebook, make sure the **Snowflake Python connector with pandas support** is installed.

You can install it using one of the following:

- If you're working inside the repo:
  ```bash
  pip install -e ".[snowflake]"
  ```

- Or install the connector directly
  ```bash
  pip install "snowflake-connector-python[pandas]"
  ```

### 📦 Importing Required Libraries

In [1]:
import pydough
import datetime
import pandas as pd

### 🔐 Loading Credentials and Connecting to Snowflake

This block performs two main tasks:

1. **Load credentials from a local JSON file**:
   - The `creds.json` file contains your Snowflake login details like username, password, account name, database, schema, and warehouse.
   - These are read using Python’s built-in `json` module and stored in variables.

2. **Connect to Snowflake using PyDough**:
   - `pydough.active_session.load_metadata_graph(...)` loads a metadata graph that maps your Snowflake schema (used for query planning or optimizations).
   - `connect_database(...)` uses the loaded credentials to establish a live connection to your Snowflake database.

📌 Make sure:
- The `creds.json` file exists and contains all the required keys.
- The metadata graph path points to a valid JSON file that represents your schema.


In [2]:
import json

# Step 1: Load credentials from a JSON file
path_to_creds = "./creds.json"
with open(path_to_creds) as f:
    creds = json.load(f)

sf_username = creds["SF_USERNAME"]
sf_password = creds["SF_PASSWORD"]
sf_account = creds["SF_ACCOUNT"]
sf_tpch_db = creds["SF_DATABASE"]
sf_tpch_schema = creds["SF_SCHEMA"]
sf_warehouse = creds["SF_WH"]

# Step 2: Load a sample metadata graph and connect PyDough to Snowflake
pydough.active_session.load_metadata_graph("../../tests/test_metadata/snowflake_sample_graphs.json", "TPCH")
pydough.active_session.connect_database("snowflake", 
        user=sf_username,
        password=sf_password,
        account=sf_account,
        warehouse=sf_warehouse,
        database=sf_tpch_db,
        schema=sf_tpch_schema,
)

DatabaseContext(connection=<pydough.database_connectors.database_connector.DatabaseConnection object at 0x10f55c9b0>, dialect=<DatabaseDialect.SNOWFLAKE: 'snowflake'>)

### 🔌 Enabling PyDough's Jupyter Magic Commands

This line loads the `pydough.jupyter_extensions` module, which adds **custom magic commands** (like `%%pydough`) to the notebook.

These magic commands allow you to:
- Write PyDough directly in notebook cells using `%%pydough`
- Automatically render results

This is a Jupyter-specific feature — the `%load_ext` command dynamically loads these extensions into your current notebook session.


In [3]:
%load_ext pydough.jupyter_extensions

### 📊 TPC-H Query 1 Using PyDough DSL

This cell runs **TPC-H Query 1** using PyDough's Python-style DSL instead of raw SQL.

The query computes summary statistics (like sums, averages, and counts) for orders, grouped by return flag and line status, and filtered by a shipping date cutoff.

Finally, `pydough.to_df(output)` converts and prints the result as a Pandas DataFrame for easy inspection and analysis in Python.


In [4]:
%%pydough
# TPCH Q1
output = (
        lines.WHERE((ship_date <= datetime.date(1998, 12, 1)))
        .PARTITION(name="groups", by=(return_flag, status))
        .CALCULATE(
            L_RETURNFLAG=return_flag,
            L_LINESTATUS=status,
            SUM_QTY=SUM(lines.quantity),
            SUM_BASE_PRICE=SUM(lines.extended_price),
            SUM_DISC_PRICE=SUM(lines.extended_price * (1 - lines.discount)),
            SUM_CHARGE=SUM(
                lines.extended_price * (1 - lines.discount) * (1 + lines.tax)
            ),
            AVG_QTY=AVG(lines.quantity),
            AVG_PRICE=AVG(lines.extended_price),
            AVG_DISC=AVG(lines.discount),
            COUNT_ORDER=COUNT(lines),
        )
        .ORDER_BY(L_RETURNFLAG.ASC(), L_LINESTATUS.ASC())
)
# Step 3: Execute code
pydough.to_df(output)

,L_RETURNFLAG,L_LINESTATUS,SUM_QTY,SUM_BASE_PRICE,SUM_DISC_PRICE,SUM_CHARGE,AVG_QTY,AVG_PRICE,AVG_DISC,COUNT_ORDER
0,A,F,37734107.0,56586554400.73,53758257134.8700,55909065222.827692,25.52200585,38273.12973462,0.04998530,1478493
1,N,F,991417.0,1487504710.38,1413082168.0541,1469649223.194375,25.51647192,38284.46776085,0.05009343,38854
2,N,O,76633518.0,114935210409.19,109189591897.4720,113561024263.013782,25.50201964,38248.01560906,0.05000026,3004998
3,R,F,37719753.0,56568041380.90,53741292684.6040,55889619119.831932,25.50579361,38250.85462610,0.05000941,1478870


Another way to connect to the Snowflake database is by passing a connection object directly. Here's an example:

In [5]:
import snowflake.connector as sf_connector

# Step 1: Load credentials from a JSON file
path_to_creds = "./creds.json"
with open(path_to_creds) as f:
    creds = json.load(f)

sf_username = creds["SF_USERNAME"]
sf_password = creds["SF_PASSWORD"]
sf_account = creds["SF_ACCOUNT"]
sf_tpch_db = creds["SF_DATABASE"]
sf_tpch_schema = creds["SF_SCHEMA"]
sf_warehouse = creds["SF_WH"]

# Step 2: Create a Snowflake connection object
connection: sf_connector.connection.SnowflakeConnection = sf_connector.connect(
            user=sf_username,
            password=sf_password,
            account=sf_account,
            warehouse=sf_warehouse,
            database=sf_tpch_db,
            schema=sf_tpch_schema
        )

# Step 3: Load a sample metadata graph and connect PyDough to Snowflake
pydough.active_session.load_metadata_graph("../../tests/test_metadata/snowflake_sample_graphs.json", "TPCH")
conn_db = pydough.active_session.connect_database("snowflake", connection=connection)
pydough.active_session.database = conn_db

# Step 4: Execute code
pydough.to_df(output)

,L_RETURNFLAG,L_LINESTATUS,SUM_QTY,SUM_BASE_PRICE,SUM_DISC_PRICE,SUM_CHARGE,AVG_QTY,AVG_PRICE,AVG_DISC,COUNT_ORDER
0,A,F,37734107.0,56586554400.73,53758257134.8700,55909065222.827692,25.52200585,38273.12973462,0.04998530,1478493
1,N,F,991417.0,1487504710.38,1413082168.0541,1469649223.194375,25.51647192,38284.46776085,0.05009343,38854
2,N,O,76633518.0,114935210409.19,109189591897.4720,113561024263.013782,25.50201964,38248.01560906,0.05000026,3004998
3,R,F,37719753.0,56568041380.90,53741292684.6040,55889619119.831932,25.50579361,38250.85462610,0.05000941,1478870


Next, use the same connection outside PyDough to inspect the last query executed.

In [6]:
cur = conn_db.connection.cursor


last_query_id = cur.sfqid
cur.execute(f"""
    SELECT query_text
    FROM TABLE(INFORMATION_SCHEMA.QUERY_HISTORY())
    WHERE query_id = '{last_query_id}'
""")
last_query_text = cur.fetchone()[0]
print("Last query text:", last_query_text)

Last query text: SELECT
  l_returnflag AS L_RETURNFLAG,
  l_linestatus AS L_LINESTATUS,
  COALESCE(SUM(l_quantity), 0) AS SUM_QTY,
  COALESCE(SUM(l_extendedprice), 0) AS SUM_BASE_PRICE,
  COALESCE(SUM(l_extendedprice * (
    1 - l_discount
  )), 0) AS SUM_DISC_PRICE,
  COALESCE(SUM(l_extendedprice * (
    1 - l_discount
  ) * (
    1 + l_tax
  )), 0) AS SUM_CHARGE,
  AVG(l_quantity) AS AVG_QTY,
  AVG(l_extendedprice) AS AVG_PRICE,
  AVG(l_discount) AS AVG_DISC,
  COUNT(*) AS COUNT_ORDER
FROM tpch_sf1.lineitem
WHERE
  l_shipdate <= CAST('1998-12-01' AS DATE)
GROUP BY
  1,
  2
ORDER BY
  1 NULLS FIRST,
  2 NULLS FIRST


Run additional queries if needed:

In [7]:
# Example: run another test query
cur.execute("SELECT CURRENT_VERSION()")
version = cur.fetchone()[0]
print("Snowflake version:", version)

# Example: Get today's date
cur.execute("SELECT CURRENT_DATE()")
today = cur.fetchone()[0]
print("Today's date:", today)

Snowflake version: 9.24.1
Today's date: 2025-08-26


Finally, close the cursor and connection:

In [8]:
cur.close()
connection.close()